In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

In [2]:
crime_data_file = pd.read_csv('crimedata.csv')
df = pd.DataFrame(crime_data_file)
df

C:\Users\HM\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,INCIDENT_NUMBER,DATE_REPORTED,DATE_OCCURED,UOR_DESC,CRIME_TYPE,NIBRS_CODE,UCR_HIERARCHY,ATT_COMP,LMPD_DIVISION,LMPD_BEAT,PREMISE_TYPE,BLOCK_ADDRESS,CITY,ZIP_CODE,ID
0,0,80-16-111411,2017-01-01 08:24:55,2016-12-30 21:00:00,TBUT OR DISP ALL OTHERS < $500,THEFT/LARCENY,23H,PART I,COMPLETED,2ND DIVISION,223,HIGHWAY / ROAD / ALLEY,1600 BLOCK DUMESNIL ST ...,LOUISVILLE,40210,1062371
1,1,80-17-000033,2017-01-01 05:20:56,2016-12-31 18:30:00,TBUT OR DISP CONTENTS FROM VEH < $500,VEHICLE BREAK-IN/THEFT,23F,PART I,COMPLETED,3RD DIVISION,331,PARKING LOT / GARAGE,4500 BLOCK GREENWOOD RD ...,LOUISVILLE,40258,1062381
2,2,80-17-000025,2017-01-01 00:57:00,2017-01-01 00:30:00,ASSAULT - 4TH DEGREE (MINOR INJURY),ASSAULT,13B,PART II,COMPLETED,7TH DIVISION,736,HOTEL / MOTEL / ETC.,6500 BLOCK SIGNATURE DR ...,LOUISVILLE,40213,1062384
3,3,80-17-000005,2017-01-01 00:20:00,2017-01-01 00:01:00,CRIMINAL MISCHIEF-3RD DEGREE,VANDALISM,290,PART II,COMPLETED,1ST DIVISION,123,PARKING LOT / GARAGE,500 BLOCK S 5TH ST ...,LOUISVILLE,40202,1062385
4,4,80-17-000037,2017-01-01 00:35:00,2017-01-01 00:15:00,CRIMINAL MISCHIEF-3RD DEGREE,VANDALISM,290,PART II,COMPLETED,7TH DIVISION,735,RESIDENCE / HOME,6100 BLOCK OVERHILL DR ...,LOUISVILLE,40229,1062386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397160,77122,80-13-072813,2013-09-23 14:01:00,2013-09-18 14:01:00,RAPE - 1ST DEGREE,SEX CRIMES,11A,PART I,COMPLETED,4TH DIVISION,423,RESIDENCE / HOME,3700 BLOCK LENTZ AVE ...,LOUISVILLE,40215,1217471
397161,77123,80-13-027653,2013-04-19 23:32:00,2013-04-19 15:00:00,RAPE - 1ST DEGREE,SEX CRIMES,11A,PART I,COMPLETED,1ST DIVISION,111,OTHER / UNKNOWN,PARK - PORTLAND WHARF PARK ...,LOUISVILLE,40212,1238339
397162,77124,80-13-016472,2013-03-08 14:13:00,2013-01-01 00:00:00,RAPE - 1ST DEGREE,SEX CRIMES,11A,PART I,COMPLETED,METRO LOUISVILLE,METRO,RESIDENCE / HOME,COMMUNITY AT LARGE,LOUISVILLE,NaN,1238340
397163,77125,80-13-090856,2013-11-25 17:00:00,2013-11-25 07:50:00,RECEIVING STOLEN PROPERTY (FIREARM),THEFT/LARCENY,280,PART II,COMPLETED,5TH DIVISION,514,RESIDENCE / HOME,2700 BLOCK ALANMEDE RD ...,LOUISVILLE,40205,1242262


In [4]:
narrowed_df = df[['DATE_REPORTED', 'DATE_OCCURED', 'CRIME_TYPE', 'PREMISE_TYPE', 'ZIP_CODE']]
narrowed_df

,DATE_REPORTED,DATE_OCCURED,CRIME_TYPE,PREMISE_TYPE,ZIP_CODE
0,2017-01-01 08:24:55,2016-12-30 21:00:00,THEFT/LARCENY,HIGHWAY / ROAD / ALLEY,40210
1,2017-01-01 05:20:56,2016-12-31 18:30:00,VEHICLE BREAK-IN/THEFT,PARKING LOT / GARAGE,40258
2,2017-01-01 00:57:00,2017-01-01 00:30:00,ASSAULT,HOTEL / MOTEL / ETC.,40213
3,2017-01-01 00:20:00,2017-01-01 00:01:00,VANDALISM,PARKING LOT / GARAGE,40202
4,2017-01-01 00:35:00,2017-01-01 00:15:00,VANDALISM,RESIDENCE / HOME,40229
...,...,...,...,...,...
397160,2013-09-23 14:01:00,2013-09-18 14:01:00,SEX CRIMES,RESIDENCE / HOME,40215
397161,2013-04-19 23:32:00,2013-04-19 15:00:00,SEX CRIMES,OTHER / UNKNOWN,40212
397162,2013-03-08 14:13:00,2013-01-01 00:00:00,SEX CRIMES,RESIDENCE / HOME,NaN
397163,2013-11-25 17:00:00,2013-11-25 07:50:00,THEFT/LARCENY,RESIDENCE / HOME,40205


In [8]:
data = narrowed_df.rename(columns={"DATE_REPORTED": "Date Reported", "DATE_OCCURED": "Date Occurred",
                                  "CRIME_TYPE": "Crime Type", "PREMISE_TYPE": "Premise Type",
                                  "ZIP_CODE": "Zip Code"})

data

,Date Reported,Date Occurred,Crime Type,Premise Type,Zip Code,Year
0,2017-01-01 08:24:55,2016-12-30 21:00:00,THEFT/LARCENY,HIGHWAY / ROAD / ALLEY,40210,2016
1,2017-01-01 05:20:56,2016-12-31 18:30:00,VEHICLE BREAK-IN/THEFT,PARKING LOT / GARAGE,40258,2016
2,2017-01-01 00:57:00,2017-01-01 00:30:00,ASSAULT,HOTEL / MOTEL / ETC.,40213,2017
3,2017-01-01 00:20:00,2017-01-01 00:01:00,VANDALISM,PARKING LOT / GARAGE,40202,2017
4,2017-01-01 00:35:00,2017-01-01 00:15:00,VANDALISM,RESIDENCE / HOME,40229,2017
...,...,...,...,...,...,...
397160,2013-09-23 14:01:00,2013-09-18 14:01:00,SEX CRIMES,RESIDENCE / HOME,40215,2013
397161,2013-04-19 23:32:00,2013-04-19 15:00:00,SEX CRIMES,OTHER / UNKNOWN,40212,2013
397162,2013-03-08 14:13:00,2013-01-01 00:00:00,SEX CRIMES,RESIDENCE / HOME,NaN,2013
397163,2013-11-25 17:00:00,2013-11-25 07:50:00,THEFT/LARCENY,RESIDENCE / HOME,40205,2013


In [14]:
# start stop and step variables
start, stop, step = 0, -15, 1

# converting to string data type 
data["Date Occurred"]= data["Date Occurred"].astype(str) 

# slicing till year element 
data["Year Occured"]= data["Date Occurred"].str.slice(start, stop, step) 

# display 
data

,Date Reported,Date Occurred,Crime Type,Premise Type,Zip Code,Year,Year Occured
0,2017-01-01 08:24:55,2016-12-30 21:00:00,THEFT/LARCENY,HIGHWAY / ROAD / ALLEY,40210,2016,2016
1,2017-01-01 05:20:56,2016-12-31 18:30:00,VEHICLE BREAK-IN/THEFT,PARKING LOT / GARAGE,40258,2016,2016
2,2017-01-01 00:57:00,2017-01-01 00:30:00,ASSAULT,HOTEL / MOTEL / ETC.,40213,2017,2017
3,2017-01-01 00:20:00,2017-01-01 00:01:00,VANDALISM,PARKING LOT / GARAGE,40202,2017,2017
4,2017-01-01 00:35:00,2017-01-01 00:15:00,VANDALISM,RESIDENCE / HOME,40229,2017,2017
...,...,...,...,...,...,...,...
397160,2013-09-23 14:01:00,2013-09-18 14:01:00,SEX CRIMES,RESIDENCE / HOME,40215,2013,2013
397161,2013-04-19 23:32:00,2013-04-19 15:00:00,SEX CRIMES,OTHER / UNKNOWN,40212,2013,2013
397162,2013-03-08 14:13:00,2013-01-01 00:00:00,SEX CRIMES,RESIDENCE / HOME,NaN,2013,2013
397163,2013-11-25 17:00:00,2013-11-25 07:50:00,THEFT/LARCENY,RESIDENCE / HOME,40205,2013,2013


In [15]:
data = data.drop(columns='Date Occurred')
data

,Date Reported,Crime Type,Premise Type,Zip Code,Year,Year Occured
0,2017-01-01 08:24:55,THEFT/LARCENY,HIGHWAY / ROAD / ALLEY,40210,2016,2016
1,2017-01-01 05:20:56,VEHICLE BREAK-IN/THEFT,PARKING LOT / GARAGE,40258,2016,2016
2,2017-01-01 00:57:00,ASSAULT,HOTEL / MOTEL / ETC.,40213,2017,2017
3,2017-01-01 00:20:00,VANDALISM,PARKING LOT / GARAGE,40202,2017,2017
4,2017-01-01 00:35:00,VANDALISM,RESIDENCE / HOME,40229,2017,2017
...,...,...,...,...,...,...
397160,2013-09-23 14:01:00,SEX CRIMES,RESIDENCE / HOME,40215,2013,2013
397161,2013-04-19 23:32:00,SEX CRIMES,OTHER / UNKNOWN,40212,2013,2013
397162,2013-03-08 14:13:00,SEX CRIMES,RESIDENCE / HOME,NaN,2013,2013
397163,2013-11-25 17:00:00,THEFT/LARCENY,RESIDENCE / HOME,40205,2013,2013


In [16]:
# start stop and step variables
start, stop, step = 0, -15, 1

# converting to string data type 
data["Date Reported"]= data["Date Reported"].astype(str) 

# slicing till year element 
data["Year Reported"]= data["Date Reported"].str.slice(start, stop, step) 

# display 
data

,Date Reported,Crime Type,Premise Type,Zip Code,Year,Year Occured,Year Reported
0,2017-01-01 08:24:55,THEFT/LARCENY,HIGHWAY / ROAD / ALLEY,40210,2016,2016,2017
1,2017-01-01 05:20:56,VEHICLE BREAK-IN/THEFT,PARKING LOT / GARAGE,40258,2016,2016,2017
2,2017-01-01 00:57:00,ASSAULT,HOTEL / MOTEL / ETC.,40213,2017,2017,2017
3,2017-01-01 00:20:00,VANDALISM,PARKING LOT / GARAGE,40202,2017,2017,2017
4,2017-01-01 00:35:00,VANDALISM,RESIDENCE / HOME,40229,2017,2017,2017
...,...,...,...,...,...,...,...
397160,2013-09-23 14:01:00,SEX CRIMES,RESIDENCE / HOME,40215,2013,2013,2013
397161,2013-04-19 23:32:00,SEX CRIMES,OTHER / UNKNOWN,40212,2013,2013,2013
397162,2013-03-08 14:13:00,SEX CRIMES,RESIDENCE / HOME,NaN,2013,2013,2013
397163,2013-11-25 17:00:00,THEFT/LARCENY,RESIDENCE / HOME,40205,2013,2013,2013


In [17]:
data = data.drop(columns='Date Reported')
data

,Crime Type,Premise Type,Zip Code,Year,Year Occured,Year Reported
0,THEFT/LARCENY,HIGHWAY / ROAD / ALLEY,40210,2016,2016,2017
1,VEHICLE BREAK-IN/THEFT,PARKING LOT / GARAGE,40258,2016,2016,2017
2,ASSAULT,HOTEL / MOTEL / ETC.,40213,2017,2017,2017
3,VANDALISM,PARKING LOT / GARAGE,40202,2017,2017,2017
4,VANDALISM,RESIDENCE / HOME,40229,2017,2017,2017
...,...,...,...,...,...,...
397160,SEX CRIMES,RESIDENCE / HOME,40215,2013,2013,2013
397161,SEX CRIMES,OTHER / UNKNOWN,40212,2013,2013,2013
397162,SEX CRIMES,RESIDENCE / HOME,NaN,2013,2013,2013
397163,THEFT/LARCENY,RESIDENCE / HOME,40205,2013,2013,2013


In [20]:
crime_type =data['Crime Type'].value_counts()
crime_type

THEFT/LARCENY               78459
DRUGS/ALCOHOL VIOLATIONS    72048
ASSAULT                     65675
OTHER                       39039
BURGLARY                    31718
VANDALISM                   28404
VEHICLE BREAK-IN/THEFT      27552
FRAUD                       19342
MOTOR VEHICLE THEFT         16359
ROBBERY                      7607
WEAPONS                      5680
DISTURBING THE PEACE         2433
SEX CRIMES                   2341
HOMICIDE                      437
ARSON                          42
DUI                            29
Name: Crime Type, dtype: int64

In [21]:
premise_type =data['Premise Type'].value_counts()
premise_type

RESIDENCE / HOME                      143164
HIGHWAY / ROAD / ALLEY                 83085
PARKING LOT / GARAGE                   31660
OTHER RESIDENCE (APARTMENT/CONDO)      27062
DEPARTMENT / DISCOUNT STORE            22180
OTHER / UNKNOWN                        17818
GROCERY / SUPERMARKET                   8161
RESTAURANT                              5604
CONVENIENCE STORE                       5555
SERVICE / GAS STATION                   5432
SPECIALTY STORE (TV, FUR, ETC)          4645
DRUG STORE/DR`S OFFICE/HOSPITAL         4641
COMMERCIAL / OFFICE BUILDING            4592
HOTEL / MOTEL / ETC.                    4116
NON-ATTACHED RESD GARAGE/SHED/BULD      3440
SCHOOL - ELEMENTARY / SECONDARY         3222
PARK / PLAYGROUND                       3182
BAR / NIGHT CLUB                        2708
BANK / SAVINGS & LOAN                   2101
GOVERNMENT / PUBLIC BUILDING            1774
CHURCH / SYNAGOGUE / TEMPLE             1390
CONSTRUCTION SITE                       1296
AUTO DEALE

In [22]:
year_group = data.groupby("Year Occured").count()
year_group

,Crime Type,Premise Type,Zip Code,Year,Year Reported
Year Occured,,,,,
2013,76888,76754,76464,76888,76888
2014,77271,77201,76815,77271,77271
2015,79434,79387,79218,79434,79434
2016,83436,83325,82792,83436,83436
2017,80136,80072,79631,80136,80136
